In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
root_path='gdrive/My Drive/PLANT DISEASE DETECTION/' 
# Proj Drive Folder 
# use print(root_path) / !ls proj_folder_path  to check

In [3]:
from google.colab import files
files.upload()

In [4]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [5]:
!kaggle datasets download -d koheidozono/newplantvillage -p '/content/gdrive/My Drive/PLANT DISEASE DETECTION/'

newplantvillage.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [7]:
from zipfile import ZipFile
with ZipFile('/content/gdrive/MyDrive/PLANT DISEASE DETECTION/newplantvillage.zip', 'r') as zipObj:
  zipObj.extractall()
  print('Done')

Done


In [8]:
classes_train = sorted(list(filter(lambda x: os.path.isdir('/content/newplantvillage/' + x), os.listdir('/content/newplantvillage'))))

In [9]:
print(classes_train) #to show names of all classes

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [10]:
!pip install split_folders

In [11]:
import splitfolders
input_folder='/content/newplantvillage'
splitfolders.ratio(input_folder, output="output", seed=1337, ratio=(.8,.2)) #default values"

Copying files: 62979 files [00:09, 6900.01 files/s]


In [12]:
batch_size=128
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1/255)
train_generator=train_datagen.flow_from_directory(
    '/content/output/train',
    target_size=(200, 200),
    batch_size=batch_size,
    classes = classes_train,
    class_mode='categorical'
)
val_datagen=ImageDataGenerator(rescale=1/255)
val_generator=val_datagen.flow_from_directory(
    '/content/output/val',
    target_size=(200, 200),
    batch_size=batch_size,
    classes = classes_train,
    class_mode='categorical'
)

Found 50368 images belonging to 34 classes.
Found 12611 images belonging to 34 classes.


In [13]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 200x 200 with 3 bytes color
    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),    
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a dense layer
    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    # 5 output neurons for 5 classes with the softmax activation
    tf.keras.layers.Dense(34, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 198, 198, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 97, 97, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        1

In [15]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])

In [17]:
epochs=12
steps_per_epoch=train_generator.n//train_generator.batch_size
validation_steps=val_generator.n//val_generator.batch_size

checkpoint = keras.callbacks.ModelCheckpoint("/content/abc.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

history=model.fit(
    x=train_generator,
    y=None,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=[checkpoint]
)

Epoch 1/12
393/393 [==============================] - ETA: 0s - loss: 0.0599 - accuracy: 0.9807
Epoch 00001: val_loss improved from inf to 0.82597, saving model to /content/abc.h5
393/393 [==============================] - 114s 289ms/step - loss: 0.0599 - accuracy: 0.9807 - val_loss: 0.8260 - val_accuracy: 0.8014
Epoch 2/12
393/393 [==============================] - ETA: 0s - loss: 0.0461 - accuracy: 0.9862
Epoch 00002: val_loss improved from 0.82597 to 0.24825, saving model to /content/abc.h5
393/393 [==============================] - 113s 288ms/step - loss: 0.0461 - accuracy: 0.9862 - val_loss: 0.2483 - val_accuracy: 0.9244
Epoch 3/12
393/393 [==============================] - ETA: 0s - loss: 0.0333 - accuracy: 0.9897
Epoch 00003: val_loss did not improve from 0.24825
393/393 [==============================] - 112s 286ms/step - loss: 0.0333 - accuracy: 0.9897 - val_loss: 0.3699 - val_accuracy: 0.8968
Epoch 4/12
393/393 [==============================] - ETA: 0s - loss: 0.0368 - accur

In [19]:
from tensorflow import keras
keras.models.save_model(model, "model.h5")

In [21]:
model = tf.keras.models.load_model('/content/abc.h5')
model.evaluate(val_generator)

99/99 [==============================] - 22s 220ms/step - loss: 0.1617 - accuracy: 0.9584


[0.16170307993888855, 0.9583696722984314]

In [42]:
##-------START OF TESTING CODE----------
## run this cell for testing purpose only & after folder name add the test image path.
# ex: '/.../.../test_imgs/'+file_name ....just replace the file_name with your image file name.
## only for testing single files / images

test_images = [f for f in os.listdir('/content/test_imgs')]
dataset = np.ndarray(shape=(len(test_images), 200, 200, 3))

import cv2

for i, file_name in enumerate(test_images):
  try:
    img=cv2.imread('/content/test_imgs/' + file_name)
    img_resize=cv2.resize(img,(200,200))
    dataset[i]=np.array(img_resize)/255.0
  except:
    print(file_name)
dataset.shape

Tomato___Bacterial_spot
Grape___healthy
Orange___Haunglongbing_(Citrus_greening)
Pepper,_bell___healthy
Apple___Cedar_apple_rust
Cherry_(including_sour)___Powdery_mildew
Strawberry___healthy
Tomato___Late_blight
Tomato___Target_Spot
Apple___healthy
Grape___Esca_(Black_Measles)
Squash___Powdery_mildew
Peach___healthy
Peach___Bacterial_spot
Raspberry___healthy
Soybean___healthy
Tomato___Early_blight
Strawberry___Leaf_scorch
Tomato___Tomato_mosaic_virus
Tomato___Spider_mites Two-spotted_spider_mite
Pepper,_bell___Bacterial_spot
Tomato___Leaf_Mold
Cherry_(including_sour)___healthy
Blueberry___healthy
Potato___Late_blight
Apple___Apple_scab
Potato___healthy
Tomato___Tomato_Yellow_Leaf_Curl_Virus
Potato___Early_blight
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Apple___Black_rot
Tomato___healthy
Tomato___Septoria_leaf_spot
Grape___Black_rot


(34, 200, 200, 3)

In [44]:
results = model.predict_classes(dataset)
final_json = {}
for file_name, result in zip(test_images, results):
  final_json[file_name] = classes_train[result]


import json
with open('prediction.json', 'w') as outfile:
  json.dump(final_json, outfile)



In [48]:
from google.colab import files
files.download('/content/prediction.json')

##-------END OF TESTING CODE----------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/model.h5')
## For TFLite model
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()
open('model.tflite', 'wb').write(tflite_model)
## For TFLite optimized model
converter.optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_opt_model=converter.convert()
open('model_opt.tflite', 'wb').write(tflite_opt_model)

INFO:tensorflow:Assets written to: /tmp/tmp987zu5q8/assets


INFO:tensorflow:Assets written to: /tmp/tmp987zu5q8/assets


INFO:tensorflow:Assets written to: /tmp/tmpyx7l0fv6/assets


INFO:tensorflow:Assets written to: /tmp/tmpyx7l0fv6/assets


246176

In [64]:
from google.colab import files
files.download('model.h5')  ##generated model file
files.download('model.tflite') ##generated model tflite converted file
files.download('model_opt.tflite') ##generated model tflite converted file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>